In [1]:
import os
from pymongo import MongoClient, GEO2D
import pandas as pd
import json
import requests
from dotenv import load_dotenv
import re
from geopy.distance import geodesic 
load_dotenv()

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


True

In [5]:
'''startmongo = 'service mongod start'
command = 'mongoimport --db companies --collection companies --file Database/companies.json'
def initDB():
    os.popen(startmongo)
os.popen(startmongo)
os.popen('service mongod status')
os.popen(command)'''

In [6]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [7]:
db,coll = (list(connectCollection('companies','companies')))
rest = (list(coll.find({'$and':[{'$or':[{"offices.latitude":{'$ne':None}},{"offices.longitude":{'$ne':None}}]},{'deadpooled_year':None}]})))
value =  rest[0].get('total_money_raised')

In [8]:
#q = {'$or':[{"offices.latitude":{'$ne':'null'}},{"offices.longitude":{'$ne':'null'}}]}
rest = (list(coll.find({'$and':[{'$or':[{"offices.latitude":{'$ne':None}},{"offices.longitude":{'$ne':None}}]},{'deadpooled_year':None}]})))
clean_db = []
for i in range(len(rest)):
    value = rest[i].get('total_money_raised') 
    valueM = re.findall(r'\d+[?\.\d]?\d',value)
    if valueM == []:
        valueM = [0]
    diction ={
        'name':rest[i]['name'],
        'category':rest[i]['category_code'],
        #'deadpooled':rest[i]['deadpooled_year'],
        'value': {
            'value':float(valueM[0]),
            'qty':value[-1]
        }
        
    }
    for ind in range(len(rest[i]['offices'])):
        if rest[i]['offices']:
            diction[f'Location-{ind+1}'] = {
                'type':'Point',
                'coordinates':[float(rest[i]['offices'][ind]['longitude']), float(rest[i]['offices'][ind]['latitude'])]
            }
        
    
    clean_db.append(diction)

In [9]:
with open('Database/database_clean.json', 'w') as fp:
    json.dump(clean_db, fp)

In [10]:
#command = 'mongoimport --db companies --collection companies_clean --jsonArray --file database_clean.json'
#os.popen(command)

In [20]:
load_dotenv()
db,coll_clean = (list(connectCollection('companies','companies_clean2')))

In [21]:
def gsearch(lat,lon,radius,query):
    URL = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
    params={
        'name':str(query),
        'location':'{},{}'.format(lat,lon),
        'radius':str(radius),
        'key':os.getenv('GOOGLE_KEY'),
        }
    res=requests.get(URL, params=params).json()
    return res

In [22]:
def geolocation(data):
    list_places = []
    for i in range(len(data['results'])):
        diction={}
        lat = data['results'][0]['geometry']['location']['lat']
        lng = data['results'][0]['geometry']['location']['lng']
        diction = {
            'type':'Point',
            'coordinates':[lng,lat]
            }
        list_places.append(diction)
    return list_places

In [23]:
def getnearbyplaces(coll,radius,query,init = 0):
    coll_list = (list(coll.find({})))
    try:
        init = init
        for i in range(init,len(coll_list)):
            print(f'enterprise{i}')
            collitem =  coll_list[i][f'Location-1']
            lat = collitem['coordinates'][1]
            lon = collitem['coordinates'][0]
            data = gsearch(lat,lon,radius,query)
            geoloc = geolocation(data)
            locations = geoloc
            places = len(geoloc)
            value = {"$set":{'Location-1':{'type':'Point','coordinates':[float(lat), float(lon)],query:{'locations':locations,'Nº':places}}}}
            coll.update_one(coll_list[i],value)
    except KeyError as e:
        init = i+1
        print(f'Error,{e}')
        return getnearbyplaces(coll_clean,500,'starbucks',init)

In [24]:
#getnearbyplaces(coll_clean,500,'starbucks',13321)

In [29]:
coll_list = (list(coll_clean.find({})))
display(coll_list[250])




{'_id': ObjectId('5dd51184114b796302086476'),
 'name': 'BlueLithium',
 'category': 'advertising',
 'value': {'value': 11.5, 'qty': 'M'},
 'Location-1': {'type': 'Point',
  'coordinates': [37.32052, -121.948399],
  'starbucks': {'locations': [{'type': 'Point',
     'coordinates': [-121.949113, 37.320964]},
    {'type': 'Point', 'coordinates': [-121.949113, 37.320964]}],
   'Nº': 2}}}

In [45]:
transport = pd.read_csv("Database/airports-extended.csv")

In [46]:
transport = transport[transport.Type!= 'unknown']
transport

,Name,Lat,Lng,Type
0,Goroka Airport,-6.081690,145.391998,airport
1,Madang Airport,-5.207080,145.789001,airport
2,Mount Hagen Kagamuga Airport,-5.826790,144.296005,airport
3,Nadzab Airport,-6.569803,146.725977,airport
4,Port Moresby Jacksons International Airport,-9.443380,147.220001,airport
...,...,...,...,...
10651,Duisburg Hauptbahnhof,51.429402,6.774830,station
10657,Bamburi Airport,-3.979444,39.731388,airport
10658,Pioltello-Limito Railway Station,45.486111,9.329444,station
10659,Verdello-Dalmine Railway Station,45.605556,9.619722,station


In [47]:
geocode = []
for i in range(len(transport["Lat"])):
    lat = transport["Lat"].iloc[i]
    lng = transport["Lng"].iloc[i]
    diction = {
            'type':'Point',
            'coordinates':[lng,lat]
            }
    geocode.append(diction)

In [48]:
transport['geocode'] = geocode

In [49]:
from geopy.distance import geodesic 
def getdistance(location1,location2):
    return geodesic(location1, location2).km

In [51]:
db,coll = (list(connectCollection('companies','companies_clean2')))

In [52]:
transport = pd.read_csv("Database/airports-extended.csv")
transport = transport[transport.Type!= 'unknown']
geocode = []
for i in range(len(transport["Lat"])):
    lat = transport["Lat"].iloc[i]
    lng = transport["Lng"].iloc[i]
    diction = {
            'type':'Point',
            'coordinates':[lng,lat]
            }
    geocode.append(diction)
transport['geocode'] = geocode

In [53]:
def getdistance(location1,location2):
        return geodesic(location1, location2).km

In [54]:
coll_list = (list(coll.find({})))

In [55]:
data = pd.DataFrame(coll_list)
data = data[['_id','name','value','Location-1']]
data.dropna(subset=['Location-1'], inplace=True)
display(data.head())

,_id,name,value,Location-1
0,5dd51184114b79630208637c,Flektor,"{'value': 0.0, 'qty': '0'}","{'type': 'Point', 'coordinates': [-118.379768,..."
1,5dd51184114b79630208637d,Fox Interactive Media,"{'value': 0.0, 'qty': '0'}","{'type': 'Point', 'coordinates': [-118.39417, ..."
2,5dd51184114b79630208637e,Gizmoz,"{'value': 18.1, 'qty': 'M'}","{'type': 'Point', 'coordinates': [-122.169472,..."
3,5dd51184114b79630208637f,Scribd,"{'value': 25.8, 'qty': 'M'}","{'type': 'Point', 'coordinates': [-122.404052,..."
4,5dd51184114b796302086380,Facebook,"{'value': 2.4, 'qty': 'B'}","{'type': 'Point', 'coordinates': [-122.151801,..."


In [56]:
liststarbacks=[]
for i in range(len(data['Location-1'])):
    liststarbacks.append(data['Location-1'].iloc[i]['starbucks'])
data['Starbucks'] = liststarbacks
display(data.head())

,_id,name,value,Location-1,Starbucks
0,5dd51184114b79630208637c,Flektor,"{'value': 0.0, 'qty': '0'}","{'type': 'Point', 'coordinates': [-118.379768,...","{'locations': [], 'Nº': 0}"
1,5dd51184114b79630208637d,Fox Interactive Media,"{'value': 0.0, 'qty': '0'}","{'type': 'Point', 'coordinates': [-118.39417, ...","{'locations': [], 'Nº': 0}"
2,5dd51184114b79630208637e,Gizmoz,"{'value': 18.1, 'qty': 'M'}","{'type': 'Point', 'coordinates': [-122.169472,...","{'locations': [], 'Nº': 0}"
3,5dd51184114b79630208637f,Scribd,"{'value': 25.8, 'qty': 'M'}","{'type': 'Point', 'coordinates': [-122.404052,...","{'locations': [], 'Nº': 0}"
4,5dd51184114b796302086380,Facebook,"{'value': 2.4, 'qty': 'B'}","{'type': 'Point', 'coordinates': [-122.151801,...","{'locations': [], 'Nº': 0}"


In [57]:
geoindex = []
for i in range(len(data['Location-1'])):
    diction_2 = {
        'type':'Point',
        'coordinates':[data['Location-1'].iloc[i]['coordinates'][0],data['Location-1'].iloc[i]['coordinates'][1]]
    }
    geoindex.append(diction_2)
data['Location-1']=geoindex

In [60]:
db,coll_3 = connectCollection('companies','companies_clean_3')
for i in range(len(data['Location-1'])):
    latitud = data['Location-1'].iloc[i]['coordinates'][1]
    longitud = data['Location-1'].iloc[i]['coordinates'][0]
    if latitud > 90 or latitud < -90:
        data['Location-1'].iloc[i]['coordinates'][1]=longitud
        data['Location-1'].iloc[i]['coordinates'][0]=latitud
        print(f'Error, latitud = {latitud}, longitud = {longitud}')

In [61]:
db,coll_airports = connectCollection('companies','airports')
transport = transport[['Name','Type','geocode']]
for i in range(len(transport['geocode'])):
    latitud = transport['geocode'].iloc[i]['coordinates'][1]
    longitud = transport['geocode'].iloc[i]['coordinates'][0]
    if latitud > 90. or latitud < -90.:
        transport['geocode'].iloc[i]['coordinates'][1]=latitud/1000
        print(f'Error, latitud = {latitud}')
    if longitud > 180. or longitud < -180.:
        transport['geocode'].iloc[i]['coordinates'][0]=longitud/1000
        print(f'Error,longitud = {longitud}')

In [ ]:
'''
coll_3.create_index([('Location-1', pymongo.GEOSPHERE)])
coll_airports.insert_many(transport.to_dict('record'))
coll_airports.create_index ([('geocode', pymongo.GEOSPHERE)])
'''
